In [1]:
from calendar import c
import time
import json
import pandas as pd
import numpy as np
import river
import sys
from sklearn.metrics import mean_absolute_percentage_error
from river import preprocessing, tree, metrics

from kafka import KafkaConsumer
import time
from kafka import KafkaConsumer
import json
import pandas as pd
import river
import time
from river.tree import HoeffdingTreeClassifier
from river.neighbors import KNNClassifier
from river.stream import iter_pandas
from river import metrics
from river import utils

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [2]:
def print_progress(y, y_pred, date, sample_id, training_time, testing_time, m, m2):
    print(f'Samples processed: {sample_id}')
    print(f'Prevision at {date}')
    print(f'Predicted value : {y_pred:.2f}')
    print(f'Real value : {y}')
    print("MAE:",m)
    print("SMAPE:",m2)
    print("total train time:" , training_time)
    print("total testing time:",testing_time)
    

def evaluate(stream, model):

    list_of_trainings = []
    list_of_testings = []
    list_y = []
    list_ypred = []

    training_time = 0
    testing_time = 0
    metric = river.metrics.MAE()    
    metric_mape = river.metrics.SMAPE()

    for i, (x, y) in enumerate(stream):
        # Predict
        start = time.time()
        y_pred = model.predict_one(x)
        end = time.time()
        testing_time = end-start
        list_of_testings.append(testing_time)
        
        # Learn (train)
        start = time.time()
        list_y.append(y)
        list_ypred.append(y_pred)
        m = metric.update(y_pred, y).get()
        m2 = metric_mape.update(y_pred, y).get()
        model.learn_one(x, y)
        end = time.time()
        training_time =  end-start
        list_of_trainings.append(training_time)
        
    # Update metrics and results
    print_progress(y, y_pred, df.iloc[-1].name,i,sum(list_of_trainings),sum(list_of_testings), m, m2)
    return y, y_pred, m, m2

In [4]:
if __name__ == '__main__':

    name = sys.argv[1]
    topic_name = 'project'
    consumer = KafkaConsumer(topic_name, bootstrap_servers="localhost:9092",value_deserializer=lambda m: json.loads(m.decode('utf-8')))#json.dumps(m).decode('utf8'))
    col = ['open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'nb_trades',
        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
    model = (
            river.preprocessing.StandardScaler() |
            river.tree.HoeffdingTreeRegressor(
                grace_period=200,
                leaf_prediction='adaptive',
                model_selector_decay=0.9
            )
        )
    
    list_y = []
    list_ypred = []
    
    for i in consumer:
        res = json.loads(json.dumps(i.value))
        df = pd.DataFrame(res, columns=col)
        for i in col:
            df[i] = df[i].astype(float)
        df['open_time'] = df['open_time'].apply(lambda x: datetime.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
        df = df.set_index('open_time')
        
        y = df.close
        X = df.drop(columns = 'close')
        y, y_pred, m, m2 = evaluate(stream=iter_pandas(X=X, y=y),
                      model=model)

        list_y.append(y)
        list_ypred.append(y_pred)
        
        print()
        print('----------------------------------------------------------------------')

Samples processed: 99
Prevision at 2023-01-12 13:14:00
Predicted value : 18220.08
Real value : 18265.15
MAE: 380.7980612920456
SMAPE: 4.0350233167814835
total train time: 0.025018930435180664
total testing time: 0.003924369812011719

----------------------------------------------------------------------
Samples processed: 99
Prevision at 2023-01-12 13:15:00
Predicted value : 18220.69
Real value : 18249.0
MAE: 23.629868365781512
SMAPE: 0.12964789973347615
total train time: 0.16176056861877441
total testing time: 0.02365422248840332

----------------------------------------------------------------------
Samples processed: 99
Prevision at 2023-01-12 13:16:00
Predicted value : 18221.04
Real value : 18250.9
MAE: 24.1461745761364
SMAPE: 0.13247913336545583
total train time: 0.021962642669677734
total testing time: 0.003825664520263672

----------------------------------------------------------------------
Samples processed: 99
Prevision at 2023-01-12 13:17:00
Predicted value : 18221.37
Real 

KeyboardInterrupt: 